In [56]:
#  Датасет: https://www.kaggle.com/c/titanic/data

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score

In [17]:
df = pd.read_csv('train.csv')
df.set_index('PassengerId');

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [15]:
df.groupby(by='Sex').mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,
female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


In [32]:
y = df['Survived']
x = df.drop('Survived', axis=1)

# эти поля не используем
x = x.drop(['Cabin', 'Name', 'Ticket'], axis=1)

# В колонке Age много пропущенных значений. Заменим их средними
x['Age'] = x['Age'].mean()

# в поле Embarked есть пропущенные значения
# заменим их модой
x['Embarked'] = x['Embarked'].fillna(x['Embarked'].mode()[0])

# посольку Sex - категориальный признак, заменим эту колонку на числа
x['Sex'] = (x['Sex'] == 'male').astype('int')
x['Sex'].value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [34]:
x = pd.get_dummies(x)

In [36]:
x['Pclass'] = x['Pclass'].astype('category')

In [38]:
x = pd.get_dummies(x)

In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.25, random_state=42)

In [43]:
lr = LogisticRegression()

In [44]:
lr.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [45]:
y_pred = lr.predict(X_valid)

In [47]:
accuracy_score(y_valid, y_pred)

0.7847533632286996

In [48]:
precision_score(y_valid, y_pred)

0.7469879518072289

In [49]:
y_proba = lr.predict_proba(X_valid)

In [55]:
y_proba[:,1].round().astype(int)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0])

In [67]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

classifier = KNeighborsClassifier(n_neighbors=5)

classifier = Pipeline([('norm', StandardScaler()), ('knn', classifier)])

classifier.fit(X_train, y_train)
y_predict = classifier.predict(X_valid)
accuracy_score(y_valid, y_predict)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/opt/conda/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.8161434977578476

In [69]:
from sklearn.svm import SVC

In [70]:
clf = SVC(kernel='linear', C=1.0)

clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [71]:
y_predict = clf.predict(X_valid)

accuracy_score(y_valid, y_predict)

0.7937219730941704

In [72]:
precision_score(y_valid, y_predict)

0.7590361445783133